## Softmax regression
Softmax regression (or multinomial logistic regression) is a generalization of logistic regression to the case where we want to handle multiple classes. In logistic regression we assumed that the labels were binary: $y^{(i)} \in \{0,1\}$. We used such a classifier to distinguish between two kinds of hand-written digits. Softmax regression allows us to handle $y^{(i)} \in \{1,\ldots,K\}$ where $K$ is the number of classes.

Recall that in logistic regression, we had a training set $\{ (x^{(1)}, y^{(1)}), \ldots, (x^{(m)}, y^{(m)}) \}$ of $m$ labeled examples, where the input features are $x^{(i)} \in \Re^{n}$. With logistic regression, we were in the binary classification setting, so the labels were $y^{(i)} \in \{0,1\}$. Our hypothesis took the form:
\begin{align*}
h_\theta(x) = \frac{1}{1+\exp(-\theta^\top x)},
\end{align*}
and the model parameters $\theta$ were trained to minimize the cost function
\begin{align*}
J(\theta) = -\left[ \sum_{i=1}^m y^{(i)} \log h_\theta(x^{(i)}) + (1-y^{(i)}) \log (1-h_\theta(x^{(i)})) \right]
\end{align*}
In the softmax regression setting, we are interested in multi-class classification (as opposed to only binary classification), and so the label $y$ can take on $K$ different values, rather than only two. Thus, in our training set $\{ (x^{(1)}, y^{(1)}), \ldots, (x^{(m)}, y^{(m)}) \}$, we now have that $y^{(i)} \in \{1, 2, \ldots, K\}$. (Note that our convention will be to index the classes starting from 1, rather than from 0.) For example, in the MNIST digit recognition task, we would have $K=10$ different classes.

Given a test input $x$, we want our hypothesis to estimate the probability that $P(y=k | x)$ for each value of $k = 1, \ldots, K$. I.e., we want to estimate the probability of the class label taking on each of the $K$ different possible values. Thus, our hypothesis will output a $K$-dimensional vector (whose elements sum to 1) giving us our $K$ estimated probabilities. Concretely, our hypothesis $h_{\theta}(x)$ takes the form:
\begin{align*}
h_\theta(x) =
\begin{bmatrix}
P(y = 1 | x; \theta) \\
P(y = 2 | x; \theta) \\
\vdots \\
P(y = K | x; \theta)
\end{bmatrix}
=
\frac{1}{ \sum_{j=1}^{K}{\exp(\theta^{(j)\top} x) }}
\begin{bmatrix}
\exp(\theta^{(1)\top} x ) \\
\exp(\theta^{(2)\top} x ) \\
\vdots \\
\exp(\theta^{(K)\top} x ) \\
\end{bmatrix}
\end{align*}
Here $\theta^{(1)}, \theta^{(2)}, \ldots, \theta^{(K)} \in \Re^{n}$ are the parameters of our model. Notice that the term $\frac{1}{ \sum_{j=1}^{K}{\exp(\theta^{(j)\top} x) } }$ normalizes the distribution, so that it sums to one.

For convenience, we will also write $\theta$ to denote all the parameters of our model. When you implement softmax regression, it is usually convenient to represent $\theta$ as a $n$-by-$K$ matrix obtained by concatenating $\theta^{(1)}, \theta^{(2)}, \ldots, \theta^{(K)}$ into columns, so that
\begin{align*}
\theta = \left[\begin{array}{cccc}| & | & | & | \\
\theta^{(1)} & \theta^{(2)} & \cdots & \theta^{(K)} \\
| & | & | & |
\end{array}\right].
\end{align*}

### Cost Function
We now describe the cost function that we’ll use for softmax regression. In the equation below, $1\{\cdot\}$ is the ”‘indicator function,”’ so that $1\{\hbox{a true statement}\}=1$, and $1\{\hbox{a false statement}\}=0$. For example, $1\{2+2=4\}$ evaluates to 1; whereas $1\{1+1=5\}$ evaluates to 0. Our cost function will be:
\begin{align*}
J(\theta) = - \left[ \sum_{i=1}^{m} \sum_{k=1}^{K}  1\left\{y^{(i)} = k\right\} \log \frac{\exp(\theta^{(k)\top} x^{(i)})}{\sum_{j=1}^K \exp(\theta^{(j)\top} x^{(i)})}\right]
\end{align*}
Notice that this generalizes the logistic regression cost function, which could also have been written:
\begin{align*}
J(\theta) &= - \left[ \sum_{i=1}^m   (1-y^{(i)}) \log (1-h_\theta(x^{(i)})) + y^{(i)} \log h_\theta(x^{(i)}) \right] \\
&= - \left[ \sum_{i=1}^{m} \sum_{k=0}^{1} 1\left\{y^{(i)} = k\right\} \log P(y^{(i)} = k | x^{(i)} ; \theta) \right]
\end{align*}
The softmax cost function is similar, except that we now sum over the $K$ different possible values of the class label. Note also that in softmax regression, we have that $P(y^{(i)} = k | x^{(i)} ; \theta) = \frac{\exp(\theta^{(k)\top} x^{(i)})}{\sum_{j=1}^K \exp(\theta^{(j)\top} x^{(i)}) }$
We cannot solve for the minimum of $J(\theta)$ analytically, and thus as usual we’ll resort to an iterative optimization algorithm. Taking derivatives, one can show that the gradient is:
\begin{align*}
\nabla_{\theta^{(k)}} J(\theta) = - \sum_{i=1}^{m}{ \left[ x^{(i)} \left( 1\{ y^{(i)} = k\}  - P(y^{(i)} = k | x^{(i)}; \theta) \right) \right]  }
\end{align*}
Recall the meaning of the $\nabla_{\theta^{(k)}}$ notation. In particular, $\nabla_{\theta^{(k)}} J(\theta)$ is itself a vector, so that its $j^{th}$ element is $\frac{\partial J(\theta)}{\partial \theta_{lk}}$ the partial derivative of $J(\theta)$ with respect to the $j^{th}$ element of $\theta^{(k)}$.

Armed with this formula for the derivative, one can then plug it into a standard optimization package and have it minimize $J(\theta)$. When implementing softmax regression, we will typically use a modified version of the cost function described above;
specifically, one that incorporates weight decay.  We describe the motivation and details below.

### Properties of softmax regression parametrization
Softmax regression has an unusual property that it has a “redundant” set of parameters. To explain what this means, suppose we take each of our parameter vectors $\theta^{(j)}$, and subtract some fixed vector $\psi$ from it, so that every $\theta^{(j)}$ is now replaced with $\theta^{(j)} - \psi$ (for every $j=1, \ldots, k$). Our hypothesis now estimates the class label probabilities as.
\begin{align*}
P(y^{(i)} = k | x^{(i)} ; \theta)
&= \frac{\exp((\theta^{(k)}-\psi)^\top x^{(i)})}{\sum_{j=1}^K \exp( (\theta^{(j)}-\psi)^\top x^{(i)})}  \\
&= \frac{\exp(\theta^{(k)\top} x^{(i)}) \exp(-\psi^\top x^{(i)})}{\sum_{j=1}^K \exp(\theta^{(j)\top} x^{(i)}) \exp(-\psi^\top x^{(i)})} \\
&= \frac{\exp(\theta^{(k)\top} x^{(i)})}{\sum_{j=1}^K \exp(\theta^{(j)\top} x^{(i)})}.
\end{align*}
In other words, subtracting $\psi$ from every $\theta^{(j)}$ does not affect our hypothesis’ predictions at all! This shows that softmax regression’s parameters are “redundant.” More formally, we say that our softmax model is ”‘overparameterized,”’ meaning that for any hypothesis we might fit to the data, there are multiple parameter settings that give rise to exactly the same hypothesis function $h_\theta$ mapping from inputs $x$ to the predictions.

Further, if the cost function $J(\theta)$ is minimized by some setting of the parameters $(\theta^{(1)}, \theta^{(2)},\ldots, \theta^{(k)})$, then it is also minimized by $(\theta^{(1)} - \psi, \theta^{(2)} - \psi,\ldots,
\theta^{(k)} - \psi)$ for any value of $\psi$. Thus, the minimizer of $J(\theta)$ is not unique. (Interestingly, $J(\theta)$ is still convex, and thus gradient descent will not run into local optima problems. But the Hessian is singular/non-invertible, which causes a straightforward implementation of Newton’s method to run into numerical problems.)

Notice also that by setting $\psi = \theta^{(K)}$, one can always replace $\theta^{(K)}$ with $\theta^{(K)} - \psi = \vec{0}$ (the vector of all 0’s), without affecting the hypothesis. Thus, one could “eliminate” the vector of parameters $\theta^{(K)}$ (or any other $\theta^{(k)}$, for any single value of $k$), without harming the representational power of our hypothesis. Indeed, rather than optimizing over the $K\cdot n$ parameters $(\theta^{(1)}, \theta^{(2)},\ldots, \theta^{(K)})$ (where $\theta^{(k)} \in \Re^{n}$), one can instead set $\theta^{(K)} = \vec{0}$ and optimize only with respect to the $K \cdot n$ remaining parameters. In practice, however, it is often cleaner and simpler to implement the version which keeps
all the parameters $(\theta^{(1)}, \theta^{(2)},\ldots, \theta^{(n)})$, without
arbitrarily setting one of them to zero.  But we will
make one change to the cost function: Adding weight decay.  This will take care of
the numerical problems associated with softmax regression's overparameterized representation.

### Weight Decay
We will modify the cost function by adding a weight decay term 
$\textstyle \frac{\lambda}{2} \sum_{i=1}^k \sum_{j=0}^{n} \theta_{ij}^2$
which penalizes large values of the parameters.  Our cost function is now
\begin{align*}
J(\theta) = - \frac{1}{m} \left[ \sum_{i=1}^{m} \sum_{j=1}^{k} 1\left\{y^{(i)} = j\right\} \log \frac{e^{\theta^{(j)\top} x^{(i)}}}{\sum_{l=1}^k e^{ \theta^{(l)\top} x^{(i)} }}  \right]
              + \frac{\lambda}{2} \sum_{i=1}^k \sum_{j=0}^n \theta_{ij}^2
\end{align*}

With this weight decay term (for any $\lambda > 0$), the cost function
$J(\theta)$ is now strictly convex, and is guaranteed to have a
unique solution.  The Hessian is now invertible, and because $J(\theta)$ is 
convex, algorithms such as gradient descent, L-BFGS, etc. are guaranteed
to converge to the global minimum. To apply an optimization algorithm, we also need the derivative of this
new definition of $J(\theta)$.  One can show that the derivative is:
\begin{align*}
\nabla_{\theta^{(j)}} J(\theta) = - \frac{1}{m} \sum_{i=1}^{m}{ \left[ x^{(i)} ( 1\{ y^{(i)} = j\}  - P(y^{(i)} = j | x^{(i)}; \theta) ) \right]  } + \lambda \theta^{(j)}
\end{align*}

By minimizing $J(\theta)$ with respect to $\theta$, we will have a working implementation of softmax regression.
\subsection{Relationship to Logistic Regression}
In the special case where $K = 2$, one can show that softmax regression reduces to logistic regression. This shows that softmax regression is a generalization of logistic regression. Concretely, when $K=2$, the softmax regression hypothesis outputs
\begin{align*}
h_\theta(x) &=
\frac{1}{ \exp(\theta^{(1)\top}x)  + \exp( \theta^{(2)\top} x^{(i)} ) }
\begin{bmatrix}
\exp( \theta^{(1)\top} x ) \\
\exp( \theta^{(2)\top} x )
\end{bmatrix}
\end{align*}
Taking advantage of the fact that this hypothesis is overparameterized and setting $\psi = \theta^{(2)}$, we can subtract $\theta^{(2)}$ from each of the two parameters, giving us
\begin{align*}
h(x) &=
\frac{1}{ \exp( (\theta^{(1)}-\theta^{(2)})^\top x^{(i)} ) + \exp(\vec{0}^\top x) }
\begin{bmatrix}
\exp( (\theta^{(1)}-\theta^{(2)})^\top x )
\exp( \vec{0}^\top x ) \\
\end{bmatrix} \\
&=
\begin{bmatrix}
\frac{1}{ 1 + \exp( (\theta^{(1)}-\theta^{(2)})^\top x^{(i)} ) } \\
\frac{\exp( (\theta^{(1)}-\theta^{(2)})^\top x )}{ 1 + \exp( (\theta^{(1)}-\theta^{(2)})^\top x^{(i)} ) }
\end{bmatrix} \\
&=
\begin{bmatrix}
\frac{1}{ 1  + \exp( (\theta^{(1)}-\theta^{(2)})^\top x^{(i)} ) } \\
1 - \frac{1}{ 1  + \exp( (\theta^{(1)}-\theta^{(2)})^\top x^{(i)} ) } \\
\end{bmatrix}
\end{align*}
Thus, replacing $\theta^{(2)}-\theta^{(1)}$ with a single parameter vector $\theta'$, we find that softmax regression predicts the probability of one of the classes as $\frac{1}{ 1  + \exp(- (\theta')^\top x^{(i)} ) }$, and that of the other class as $1 - \frac{1}{ 1 + \exp(- (\theta')^\top x^{(i)} ) }$, same as logistic regression.

### Relationship to Logistic Regression
In the special case where k = 2, one can show that softmax regression reduces to logistic regression. This shows that softmax regression is a generalization of logistic regression. Concretely, when k = 2, the softmax regression hypothesis outputs

\begin{align*}
h_\theta(x) &= \frac{1}{ e^{\theta_1^Tx}  + e^{ \theta_2^T x^{(i)} } }
\begin{bmatrix}
&e^{ \theta_1^T x } \\&e^{ \theta_2^T x }
\end{bmatrix}
\end{align*}

Taking advantage of the fact that this hypothesis is overparameterized and setting ψ = θ1, we can subtract θ1 from each of the two parameters, giving us


Thus, replacing θ2 − θ1 with a single parameter vector θ', we find that softmax regression predicts the probability of one of the classes as $$\frac{1}{ 1  + e^{ (\theta')^T x^{(i)} } }$$, and that of the other class as $$1 - \frac{1}{ 1 + e^{ (\theta')^T x^{(i)} } }$$, same as logistic regression.


### Softmax Regression vs. $k$ Binary Classifiers
Suppose you are working on a music classification application, and there are
$k$ types of music that you are trying to recognize.  Should you use a
softmax classifier, or should you build $k$ separate binary classifiers using
logistic regression?

This will depend on whether the four classes are ''mutually exclusive.''  For example,
if your four classes are classical, country, rock, and jazz, then assuming each
of your training examples is labeled with exactly one of these four class labels,
you should build a softmax classifier with $k=4$.
(If there're also some examples that are none of the above four classes,
then you can set $k=5$ in softmax regression, and also have a fifth, "none of the above," class.)

If however your categories are has\_vocals, dance, soundtrack, pop, then the
classes are not mutually exclusive; for example, there can be a piece of pop
music that comes from a soundtrack and in addition has vocals.  In this case, it
would be more appropriate to build 4 binary logistic regression classifiers. 
This way, for each new musical piece, your algorithm can separately decide whether
it falls into each of the four categories.

Now, consider a computer vision example, where you're trying to classify images into
three different classes.  (i) Suppose that your classes are indoor\_scene,
outdoor\_urban\_scene, and outdoor\_wilderness\_scene.  Would you use softmax regression
or three logistic regression classifiers?  (ii) Now suppose your classes are
indoor\_scene, black\_and\_white\_image, and image\_has\_people.  Would you use softmax
regression or multiple logistic regression classifiers?

In the first case, the classes are mutually exclusive, so a softmax regression
classifier would be appropriate.  In the second case, it would be more appropriate to build
three separate logistic regression classifiers.


## Numerical Example (MNIST)

In [8]:
%matplotlib inline 
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from numpy import newaxis, c_, mat
from numpy.linalg import *
from scipy import optimize

Our cost function is now
\begin{align*}
J(\theta) = - \frac{1}{m} \left[ \sum_{i=1}^{m} \sum_{j=1}^{k} 1\left\{y^{(i)} = j\right\} \log \frac{e^{\theta^{(j)\top} x^{(i)}}}{\sum_{l=1}^k e^{ \theta^{(l)\top} x^{(i)} }}  \right]
              + \frac{\lambda}{2} \sum_{i=1}^k \sum_{j=0}^n \theta_{ij}^2
\end{align*}

In [7]:
def softmax_cost(theta, num_classes, input_size, lambda_, X, y):
    '''
    :param theta:
    :param num_classes: the number of classes
    :param input_size: the size N of input vector(num of features)
    :param lambda_: weight decay parameter
    :param data: the M x N input matrix, where each column corresponds
                 a single test set
    :param labels: an M x 1 matrix containing the labels for the input data
    '''
    
    m, n = X.shape
    theta = theta.reshape(n, num_classes)
    theta_data = X.dot(theta)
    theta_data = theta_data - np.max(theta_data) # overparameterized
    
    prob_data = np.exp(theta_data) / np.exp(theta_data).sum()
    
    indicator = np.zeros((m, theta.shape[1]), dtype=np.int8)
    for i, idx in enumerate(y):
        indicator[i][idx] = 1
  
    value = np.sum((indicator.T).dot(np.log(prob_data)))

    cost = (-1 / m) * value + (lambda_ / 2) * np.sum(theta * theta)

    grad = (-1 / m) * X.T.dot(indicator - prob_data) + lambda_ * theta

    return cost, grad.flatten()
    

Our hypothesis now estimates the class label probabilities as.
\begin{align*}
P(y^{(i)} = k | x^{(i)} ; \theta)
&= \frac{\exp(\theta^{(k)\top} x^{(i)})}{\sum_{j=1}^K \exp(\theta^{(j)\top} x^{(i)})}.
\end{align*}


In [ ]:
def softmax_predict(model, data):
    # model - model trained using softmaxTrain
    # data - the N x M input matrix, where each column data(:, i) corresponds to
    #        a single test set
    #
    # Your code should produce the prediction matrix
    # pred, where pred(i) is argmax_c P(y(c) | x(i)).

    opt_theta, input_size, num_classes = model
    opt_theta = opt_theta.reshape(input_size, num_classes)

    prod = data.dot(opt_theta)
    pred = np.exp(prod) / np.sum(np.exp(prod), axis=0)
    pred = pred.argmax(axis=0)

    return pred

The derivative of this
new definition of $J(\theta)$.  One can show that the derivative is:
\begin{align*}
\nabla_{\theta^{(j)}} J(\theta) = - \frac{1}{m} \sum_{i=1}^{m}{ \left[ x^{(i)} ( 1\{ y^{(i)} = j\}  - P(y^{(i)} = j | x^{(i)}; \theta) ) \right]  } + \lambda \theta^{(j)}
\end{align*}

In [5]:
def softmax_train(input_size, num_classes, lambda_, data, labels, options={'maxiter': 400, 'disp': True}):
    #softmaxTrain Train a softmax model with the given parameters on the given
    # data. Returns softmaxOptTheta, a vector containing the trained parameters
    # for the model.
    #
    # input_size: the size of an input vector x^(i)
    # num_classes: the number of classes
    # lambda_: weight decay parameter
    # input_data: an N by M matrix containing the input data, such that
    #            inputData(:, c) is the cth input
    # labels: M by 1 matrix containing the class labels for the
    #            corresponding inputs. labels(c) is the class label for
    #            the cth input
    # options (optional): options
    #   options.maxIter: number of iterations to train for

    # Initialize theta randomly
    theta = 0.005 * np.random.randn(num_classes * input_size)

    J = lambda x: softmax_cost(x, num_classes, input_size, lambda_, data, labels)

    result = optimize.minimize(J, theta, method='L-BFGS-B', jac=True, options=options)

    # Return optimum theta, input size & num classes
    opt_theta = result.x

    return opt_theta, input_size, num_classes
  
def featureNormalize(X):
    X_norm = X.A
 
    mu = X_norm.mean(axis=0)
    X_norm -= mu # broadcasting
 
    sigma = X_norm.std(axis=0)
    X_norm /= sigma
 
    return mat(X_norm), mu, sigma

def compute_gradient(J, theta): 
    # theta: a vector of parameters
    # J: a function that outputs a real-number. Calling y = J(theta) will return the
    # function value at theta.
    epsilon = 0.0001

    gradient = np.zeros(theta.shape)
    for i in range(theta.shape[0]):
        theta_epsilon_plus = np.array(theta, dtype=np.float64)
        theta_epsilon_plus[i] = theta[i] + epsilon
        theta_epsilon_minus = np.array(theta, dtype=np.float64)
        theta_epsilon_minus[i] = theta[i] - epsilon

        gradient[i] = (J(theta_epsilon_plus)[0] - J(theta_epsilon_minus)[0]) / (2 * epsilon)
        if i % 100 == 0:
            print ("Computing gradient for input:", i)

    return gradient

In [60]:
#Main Function
if __name__ == '__main__':
    # =================== Part 0: load dataset

    f = pd.read_csv('./Data/mnist_train.csv') 
    #f.head()
    data = np.mat(f)
    #x = np.reshape(x1, (28,28))
    #plt.imshow(x, cmap='gray')
    
    X = mat(data[:, 1:])
    y = c_[data[:, 0]]
    #idx = (y == 0).nonzero()[:1]
    #y[idx,0] = 10
    m, n = X.shape
 
    # =================== Part 1: Feature Normalization

    #X, mu, sigma = featureNormalize(X)
 
    # =================== Part 2: Initialise constants and parameters

    featureSize = n # Size of input vector (MNIST images are 28x28)
    k = 10     # Number of classes (MNIST images fall into 10 classes)

    lambda_ = 1e-4 # Weight decay parameter
    
    theta = np.random.randn(n * k, 1) * 0.005
    
    # =================== Part 3: Implement softmaxCost

    cost, grad = softmax_cost(theta, k, featureSize, lambda_, X, y)
    
    # =================== Part 5: Gradient checking
    #
    #  As with any learning algorithm, you should always check that your
    #  gradients are correct before learning the parameters.
    #
     debug = False

    images = X
    labels = y

    if debug:
        J = lambda theta_: softmax.softmax_cost(theta_, num_classes, featureSize, lambda_, X, y)

        num_grad = gradient.compute_gradient(J, theta)

        # Use this to visually compare the gradients side by side
        print(num_grad, grad)

        # Compare numerically computed gradients with the ones obtained from backpropagation
        diff = np.linalg.norm(num_grad - grad) / np.linalg.norm(num_grad + grad)
        print (diff)
        print ("Norm of the difference between numerical and analytical num_grad (should be < 1e-7)\n\n")

    # ================= Part 6: Learning parameters
    #
    #  Once you have verified that your gradients are correct,
    #  you can start training your softmax regression code using softmaxTrain

    options_ = {'maxiter': 100, 'disp': True}
    opt_theta, input_size, num_classes = softmax_train(input_size, num_classes, lambda_, input_data, labels, options_)

    # ================ Part 7: Testing
    #
    #  You should now test your model against the test images.
    #  To do this, you will first need to write softmaxPredict
    #  (in softmaxPredict.m), which should return predictions
    #  given a softmax model and the input data.
    
    file_ = pd.read_csv('./Data/mnist_test.csv') 
    data = np.mat(file_)
    
    test_images = mat(data[:, 1:])
    test_labels = c_[data[:, 0]]

    predictions = softmax_predict((opt_theta, input_size, num_classes), test_images)
    print "Accuracy: {0:.2f}%".format(100 * np.sum(predictions == test_labels, dtype=np.float64) / test_labels.shape[0])


477.719057553
(1, 7840)
